<h1>Model - Number of Trips<h1>

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Normalization

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import sum,avg,max,min,mean,count
import numpy as np

In [3]:
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "8g")
    .config("spark.sql.parquet.enableVectorizedReader", False)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/08/16 19:16:10 WARN Utils: Your hostname, Sens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.11 instead (on interface en0)
22/08/16 19:16:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 19:16:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/16 19:16:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sdf = spark.read.parquet('../data/curated/combined_data')

In [5]:
df = sdf.groupby('Date', 'Hour', 'PU_Location_ID').agg(avg("Temperature_C").alias("Temperature_C"), \
                                                           avg("Humidity_%").alias("Humidity_%"), \
                                                           avg("Speed_kmh").alias("Speed_kmh"), \
                                                           avg("Precip_Rate_mm").alias("Precip_rate_mm"), \
                                                           avg("Driver_pay").alias("Avg_driver_pay"), \
                                                           avg("Day_of_week").alias("Day_of_week"), \
                                                           count('Temperature_C').alias("Num_trips")).toPandas()

In [6]:
df = df.sort_values('Date', axis = 0)

In [7]:
df = df.drop('Date', axis = 1)

In [8]:
df = pd.get_dummies(df, columns=['Hour', 'PU_Location_ID', 'Day_of_week'])

In [9]:
TARGET_COLS = ['Num_trips']

train, test = train_test_split(df, train_size=0.8, shuffle = False)

X_train, y_train = train.drop(TARGET_COLS, axis=1).drop('Avg_driver_pay', axis=1), train[TARGET_COLS]
X_test, y_test = test.drop(TARGET_COLS, axis=1).drop('Avg_driver_pay', axis=1), test[TARGET_COLS]

In [10]:
norm_layer = Normalization()
norm_layer.adapt(X_train)

Metal device set to: Apple M1


2022-08-16 19:18:00.616671: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-16 19:18:00.617001: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-08-16 19:18:05.775225: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-16 19:18:05.892331: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-16 19:18:05.911532: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [11]:
model = keras.Sequential(
    [   
        norm_layer,
        Dense(10, activation='relu'),
        Dense(1, activation='relu')
    ]
)

In [12]:
model.compile(
    optimizer='adam',
    loss='MSE'
)

In [13]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=16,
    validation_split=0.25,
    epochs=8
)

Epoch 1/8


2022-08-16 19:22:11.763233: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


81606/81606 [==============================] - ETA: 0s - loss: 2774.2661

2022-08-16 19:27:11.209557: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


81606/81606 [==============================] - 358s 4ms/step - loss: 2774.2661 - val_loss: 2101.2178
Epoch 2/8
81606/81606 [==============================] - 356s 4ms/step - loss: 1934.3385 - val_loss: 1870.5824
Epoch 3/8
81606/81606 [==============================] - 357s 4ms/step - loss: 1853.4757 - val_loss: 1760.8907
Epoch 4/8
81606/81606 [==============================] - 357s 4ms/step - loss: 1738.9683 - val_loss: 1682.0385
Epoch 5/8
81606/81606 [==============================] - 357s 4ms/step - loss: 1656.2844 - val_loss: 1607.9918
Epoch 6/8
81606/81606 [==============================] - 358s 4ms/step - loss: 1583.1403 - val_loss: 1557.0962
Epoch 7/8
81606/81606 [==============================] - 358s 4ms/step - loss: 1551.6143 - val_loss: 1501.1436
Epoch 8/8
81606/81606 [==============================] - 357s 4ms/step - loss: 1526.2845 - val_loss: 1469.3706


In [14]:
comparison = y_test.iloc[:10].copy()
comparison.loc[:, 'prediction_num_trips'] = model.predict(X_test.head(10))
comparison

1/1 [==============================] - 0s 65ms/step


2022-08-16 20:36:30.236752: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Num_trips,prediction_num_trips
869526,489,397.478027
869527,257,260.786560
869528,154,139.096848
869529,172,171.483337
869530,69,72.226959
869531,191,220.630081
869532,31,22.376217
869533,260,239.920273
869534,149,143.006683
869535,128,156.332870
